results visualizations for paper or something idk

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import json
import math
import os
from pathlib import Path
from typing import List
import sys

import cv2
import dataclass_array as dca
import jax.numpy as jnp
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
from PIL import Image
import pandas as pd
import pycolmap
import pyrender
import trimesh
import visu3d as v3d
import yaml

sys.path.append(os.path.abspath(os.path.join("..", "bop_toolkit")))
from bop_toolkit.bop_toolkit_lib.pose_error import vsd, mssd, mspd
from bop_toolkit.bop_toolkit_lib.misc import get_symmetry_transformations
from bop_toolkit.bop_toolkit_lib.renderer import create_renderer

import burybarrel.colmap_util as cutil
from burybarrel.image import render_v3d, render_models, to_contour

In [ ]:
metricscsvpath = Path("/scratch/jeyan/barreldata/results/allmetrics.csv")
metricsdf = pd.read_csv(metricscsvpath)
metricsdf.head()

In [ ]:
metricsdf[["avg_vsd", "avg_mssd", "avg_mspd", "multiview_fitted", "pose_type", "use_icp"]].groupby(["multiview_fitted", "pose_type", "use_icp"]).mean()

In [ ]:
resrow = metricsdf.iloc[3]
datadir = Path(resrow["datadir"])
resdir = Path(resrow["resdir"])
object_name = resrow["object_name"]

with open(datadir / "gt_obj2cam.json", "rt") as f:
    gtposes = yaml.safe_load(f)
with open(datadir / "info.json", "rt") as f:
    datainfo = yaml.safe_load(f)
with open(datadir / "camera.json", "rt") as f:
    caminfo = yaml.safe_load(f)
posepath = resdir / "estimated-poses.json"
estinfopath = resdir / "reconstruction-info.json"
with open(posepath, "rt") as f:
    ests = yaml.safe_load(f)
with open(estinfopath, "rt") as f:
    estinfo = yaml.safe_load(f)

In [ ]:
ests[1]

In [ ]:
v3d.Transform.from_matrix(np.eye(4)).matrix4x4

In [ ]:
objpath = Path("/scratch/jeyan/barreldata/models3d") / object_name
mesh = trimesh.load_mesh(objpath)

In [ ]:
mesh.visual.vertex_colors = np.array([255, 0, 0, 255], dtype=np.uint8)

In [ ]:
render_models()